<a href="https://colab.research.google.com/github/nsmsm/ASL/blob/master/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_%EB%AC%B8%EC%8A%B9%EB%AA%85_(201985010061).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>